In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import pandas as pd

In [2]:
# Sigmoid activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [3]:
# Load and preprocess Air Quality dataset
def load_air_quality_data(file_path):
    data = pd.read_csv(file_path)
    features = ["PM2.5", "NO2", "CO", "Temperature", "Humidity"]
    target = "AQI_Level"
    X = data[features].values
    y = data[target].values.reshape(-1, 1)
    
    encoder = OneHotEncoder(sparse_output=False)
    y = encoder.fit_transform(y)
    
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, scaler, encoder


In [4]:
# Neural Network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output, learning_rate):
        self.error = y - output
        self.delta2 = self.error * sigmoid_derivative(output)
        self.error_hidden = np.dot(self.delta2, self.W2.T)
        self.delta1 = self.error_hidden * sigmoid_derivative(self.a1)
        self.W2 += learning_rate * np.dot(self.a1.T, self.delta2)
        self.b2 += learning_rate * np.sum(self.delta2, axis=0, keepdims=True)
        self.W1 += learning_rate * np.dot(X.T, self.delta1)
        self.b1 += learning_rate * np.sum(self.delta1, axis=0, keepdims=True)

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - output))
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)

In [5]:
# Main execution
if __name__ == "__main__":
    file_path = r"C:\Users\SWAROOP\Desktop\air_quality_dataset.csv"
    X_train, X_test, y_train, y_test, scaler, encoder = load_air_quality_data(file_path)
    nn = NeuralNetwork(input_size=5, hidden_size=6, output_size=y_train.shape[1])
    nn.train(X_train, y_train, epochs=1000, learning_rate=0.1)
    y_pred = nn.predict(X_test)
    y_test_labels = np.argmax(y_test, axis=1)
    accuracy = np.mean(y_pred == y_test_labels)
    print(f"\nTest Accuracy: {accuracy * 100:.2f}%")
    


Epoch 0, Loss: 0.2511
Epoch 100, Loss: 0.1332
Epoch 200, Loss: 0.1131
Epoch 300, Loss: 0.1190
Epoch 400, Loss: 0.1109
Epoch 500, Loss: 0.0942
Epoch 600, Loss: 0.1029
Epoch 700, Loss: 0.0799
Epoch 800, Loss: 0.0645
Epoch 900, Loss: 0.0636

Test Accuracy: 66.00%


In [7]:
 # New test data
 new_test_data = np.array([[15, 20, 0.5, 5, 30],
                               [75, 85, 2.5, 15, 50],
                               [120, 130, 5.0, 25, 60],
                               [180, 160, 7.5, 30, 75],
                               [250, 190, 9.5, 35, 85]])

# Normalize the new test data using the same scaler
new_test_data = scaler.transform(new_test_data)

# Predict the AQI level for new data
new_predictions = nn.predict(new_test_data)
 decoded_predictions = encoder.inverse_transform(np.eye(y_train.shape[1])[new_predictions])

print("\nPredictions for new test data:")
for i, pred in enumerate(decoded_predictions):
    print(f"Sample {i+1}: Predicted AQI Level - {pred[0]}")



Predictions for new test data:
Sample 1: Predicted AQI Level - Moderate
Sample 2: Predicted AQI Level - Moderate
Sample 3: Predicted AQI Level - Unhealthy for Sensitive Groups
Sample 4: Predicted AQI Level - Unhealthy
Sample 5: Predicted AQI Level - Very Unhealthy
